# Zip Code to Census Tract Conversion

##### By: Daniel Garcia Rodriguez
##### Date: 2/13/20201

## Packages Used

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

import json
import requests
import os
import math
import folium
from folium.plugins import Search
from folium.features import DivIcon

# Census Tract 2018
Click <a href='https://www2.census.gov/geo/tiger/TIGER2018/TRACT/'>here</a> for source. California state FIPS code is 06 so download file with this name: ```tl_2018_06_tract.zip.```

In [2]:
zc = ['90011','90025','90027','90032','90240']

# windows path
path = os.getcwd()+ "\\tl_2018_06_tract\\tl_2018_06_tract.shp"
tracts = gpd.read_file(path)

# Zip Code Boundaries (last updated 2020)
Click <a href="source:https://geohub.lacity.org/datasets/71b2fed2c1f24fc8850f5b7f8d2a320a_26/geoservice?geometry=-119.165%2C33.843%2C-117.430%2C34.241&orderBy=ZIPCODE&selectedAttribute=ZIP">here</a> for source.

In this section I made an API request and retrived the shape boundaries of each zip code in LA. Then I read the Json output using geopandas and saved the result into a geojson file.|

In [6]:
zipGeo = gpd.read_file(requests.get("https://maps.lacity.org/lahub/rest/services/Boundaries/MapServer/26/query?where=ZIPCODE%20%3D%20'90240'%20OR%20ZIPCODE%20%3D%20'90011'%20OR%20ZIPCODE%20%3D%20'90025'%20OR%20ZIPCODE%20%3D%20'90027'%20OR%20ZIPCODE%20%3D%20'90032'&outFields=*&outSR=4326&f=json"
).text)
zipGeo.to_file("zipCode.json", driver="GeoJSON")

# Zip to Census Tract (using 2019 4th quarter)
Crosswalk file to retrieve all census tracts that fall within or intersect the boundaries of the zip codes of interest. Used 2019 as the date since the census data used came from the American Community Survey done in 2019.

Click <a href="https://www.huduser.gov/portal/datasets/usps_crosswalk.html">here</a> for source.

In [7]:
## 2019 4th quarter used
path = os.getcwd() + "\\ZIP_TRACT_122019.xlsx"
zipToTract = pd.read_excel(path)

### Retrieving census tracts that cross the zip codes of interest. After, keeping only the matched census tracts..

In [8]:
# census tracts that cross zip codes of interest
censusTracts2019 = zipToTract[zipToTract['ZIP'].isin( list(map(int,zc)) 
                )]['TRACT'].apply(lambda x: str(x).replace('6037','')).values
# keeping only those census tracts' shapes 
tracts = tracts[tracts['TRACTCE'].isin(censusTracts2019)].reset_index(drop=True)

## Business Data
### Checking if coordinates belong to a respective tract

In [9]:
drop = ['Unnamed: 0', 'index','TIGERLine ID', 'TIGERLine ID Side', 'State Code', 'County Code','Block Code',
        'alias','phone','categories','location.address3', 'location.address2', 'location.address1', 'name',
        'location.city', 'location.zip_code', 'location.country', 'location.state', 'location.display_address',
        'price','hours', 'coordinates.latitude', 'coordinates.longitude','is_claimed','Interpolated Latitude and Longitude',
        'error.code','error.description','transactions','TIGER Address Range Match Indicator','city'
        ]

In [10]:
# Business_data csv file comes from Yelp and Google API requests made by my teammates
path = os.getcwd()+"\\business_data.csv"
df = pd.read_csv(path).drop(columns=drop)

#changing to a GeoDataFrame to create geometry series
gdf = gpd.GeoDataFrame(df, 
                      geometry=gpd.points_from_xy(df.geometry__location__lng,
                                                  df.geometry__location__lat)
                      )

# creating dummy column to iteratively assign tracts
gdf = gdf.assign(tract = np.nan)

# checking if coordinates belong to any of the census tract polygon (see tracts dataframe)
for i in range(len(tracts)):
  # for each coordinate,  is it located within x tract??
  mask = gdf.within(tracts['geometry'].iloc[i]) # true or false
    
  # must be unique (matched one time)
  if len(gdf.loc[mask,'tract'].unique())>1: 
    raise AssertionError('Row already has an assigned Tract Name. May lead to duplicates.')
    
  # assign tract name to dummy column where matches ocurred
  gdf.loc[mask,'tract'] = tracts['TRACTCE'].iloc[i]

## Checking rows that have differing census tract names. 
```tract ``` was retrieved using census geocode service whilst ```Tract Code``` was retrieved using the point-in-polygon check (see previous cells ```for``` loop)

In [23]:
diffMatch = gdf[gdf['tract'].astype(float) != gdf['Tract Code']]
print(f'Non matching census tracts: {diffMatch.shape[0]:,}')
afterNull = diffMatch[(diffMatch['tract'].notnull()) & (gdf['Tract Code'].notnull())].shape[0]
print(f'Non matching census tracts: after taking into null values into account {afterNull:,}')

Non matching census tracts: 5,842
Non matching census tracts: after taking into null values into account 2,007


C:\Users\Daniel\Anaconda3\lib\site-packages\geopandas\geodataframe.py:828: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super(GeoDataFrame, self).__getitem__(key)


Around 5.8k rows where identified as having non matching census tracts amongst the different approach to retrieving census tracts. After taking null values in the census tracts into account, we get a better estimate at the number of rows that didn't match correctly which is at around 2k. 

I went in and looked in at possible explanations for this. I found that (1) some businesses had wrong coordinates listed in their columns or(2) the coordinates retrieved from the geocoder service were not accurate as they relied on the wrong street addresses of a business. The first error applied more to the point-in-polygon check while the second error applied more to the Census geocoder service check. 

After some talk, my teammates and I decided that it was better to rely on the coordinates (point in polygon check) as our way of retrieving the census tracts of each business location. 

*Note: The Census Geocoder service performed the geocode (including the Census Tract labeling) based on the street address rather than the coordinates--the point-in-polygon retrieved census tract based on the coordinates of the business.*


## Duplicate Check
We define duplicated business as having the same google id, business name and geo coordinates.

In [24]:
subset=['Google_ID','Google_Name','geometry__location__lat','geometry__location__lng']
gdf[gdf.duplicated(subset=subset,keep=False)].sort_values('Google_ID').head(20).T

,790,11985,325,11521,11695,498,11655,458,10940,2199,11933,738,988,12183,941,12136,12137,942,12015,820
Google_ID,ChIJ--Cwb9bHwoARjH6mBJffoVI,ChIJ--Cwb9bHwoARjH6mBJffoVI,ChIJ--gBnfnIwoARR81udKW9bJ8,ChIJ--gBnfnIwoARR81udKW9bJ8,ChIJ-1Dc-XXIwoARSyYxs7oRyss,ChIJ-1Dc-XXIwoARSyYxs7oRyss,ChIJ-2khTGvJwoARNxJB_LlNH5c,ChIJ-2khTGvJwoARNxJB_LlNH5c,ChIJ-3G8tqvJwoAR6VEZjd4tKlA,ChIJ-3G8tqvJwoAR6VEZjd4tKlA,ChIJ-3U2PNHHwoARDXxlbe1fd8k,ChIJ-3U2PNHHwoARDXxlbe1fd8k,ChIJ-5aXe3jIwoARJxYhui9I-WU,ChIJ-5aXe3jIwoARJxYhui9I-WU,ChIJ-6Dkl4bIwoARA-ZEx4l7ENQ,ChIJ-6Dkl4bIwoARA-ZEx4l7ENQ,ChIJ-6Dkl4bIwoARnTxlh7bI5yY,ChIJ-6Dkl4bIwoARnTxlh7bI5yY,ChIJ-7p0ctbHwoARqjx1P4ecKpw,ChIJ-7p0ctbHwoARqjx1P4ecKpw
Google_Name,Dentist,Dentist,Drako Tax & Accounting,Drako Tax & Accounting,Abada Inc,Abada Inc,Ria Money Transfer,Ria Money Transfer,Iron works santizo,Iron works santizo,Finding a Mortgage,Finding a Mortgage,Spiritual Fellowship Church,Spiritual Fellowship Church,Munoz Tires,Munoz Tires,Cordero Tires,Cordero Tires,Simply Ink tattoos,Simply Ink tattoos
Google_Rating,NaN,NaN,4.9,4.9,5,5,1,1,NaN,NaN,NaN,NaN,NaN,NaN,4.4,4.4,5,5,4.6,4.6
Google_Total_Ratings,NaN,NaN,10,10,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,14,14,2,2,59,58
geometry__location__lat,34.0248,34.0248,33.9925,33.9925,34.0198,34.0198,34.0174,34.0174,33.9932,33.9932,34.0287,34.0287,34.022,34.022,34.0196,34.0196,34.0196,34.0196,34.0246,34.0246
geometry__location__lng,-118.259,-118.259,-118.247,-118.247,-118.271,-118.271,-118.269,-118.269,-118.259,-118.259,-118.261,-118.261,-118.262,-118.262,-118.248,-118.248,-118.248,-118.248,-118.259,-118.259
website,NaN,NaN,http://drakoincometax.com/,http://drakoincometax.com/,NaN,NaN,https://www.riamoneytransfer.com/us/,https://www.riamoneytransfer.com/us/,https://iron-works-santizo-welder.negocio.site...,https://iron-works-santizo-welder.negocio.site...,http://www.powerhomemortgage.com/contact-us.html,http://www.powerhomemortgage.com/contact-us.html,NaN,NaN,NaN,NaN,NaN,NaN,http://simplyink.com/,http://simplyink.com/
address,2177 San Pedro Street,2177 San Pedro Street,5516 Compton Avenue,5516 Compton Avenue,128 East 32nd Street,128 East 32nd Street,307 East Jefferson Boulevard,307 East Jefferson Boulevard,930 East 54th Street,930 East 54th Street,440 East Washington Boulevard,440 East Washington Boulevard,NaN,NaN,1478 East 21st Street,1478 East 21st Street,1478 East 21st Street,1478 East 21st Street,2206 San Pedro Street,2206 San Pedro Street
zipcode_area,90011,90240,90011,90240,90240,90011,90240,90011,90240,90011,90240,90011,90011,90240,90011,90240,90240,90011,90240,90011
types__001,dentist,dentist,lawyer,lawyer,NaN,NaN,finance,finance,NaN,NaN,finance,finance,church,church,car_repair,car_repair,car_repair,car_repair,store,store


In [25]:
gdf.shape[0] / gdf.duplicated(subset=subset).sum(), gdf.duplicated(subset=subset).sum(), gdf.shape[0]

(5.360232945091514, 2404, 12886)

In total we get 2,404 (5%) duplicated rows out of the total 12.8k rows. We'll go ahead and drop these rows as they may distort our analyses later down the line.

In [26]:
gdf.drop_duplicates(subset=subset).reset_index(drop=True).to_csv('FINALbusData.csv')

In [27]:
(gdf.isnull().sum() / gdf.shape[0] *100).round(2).sort_values()[::-1]


types__009                            99.91
types__008                            99.89
types__007                            99.87
types__006                            99.74
types__005                            99.45
types__004                            98.26
types__003                            91.56
rating                                72.13
url                                   72.13
is_closed                             72.13
id                                    72.13
review_count                          72.13
types__002                            61.87
website                               47.56
Google_Total_Ratings                  47.29
Google_Rating                         47.29
types__001                            30.64
is_food_service                       30.64
TIGER Match Type                      29.18
TIGER Output Address                  29.18
Tract Code                            29.18
address                                3.53
tract                           

Lastly, it should be noted that our business dataset contained several columns with missing values, in particular for the Yelp variables (ie. review_count). For the purposes of this project we'll leave the null values as is and will defintely be noted as one of the limitations of our project.

----

# Retrieving Corresponding Census Tract For The Business Listings Data Source

### Saving filtered census tracts

In [45]:
path = os.path.join(os.getcwd() , 'filteredTracts')
if not os.path.isdir(path):
    os.makedirs(path)
    
tracts = tracts[tracts['COUNTYFP'] !=  '055'] # keep only LA county
tracts['TRACTCE'] = tracts['TRACTCE'].astype(float) #convert to float 
tracts.to_file(os.path.join(path,'tracts.shp')) # save as a shapefile

In [12]:
parentDir = os.path.dirname(os.getcwd()) #parent directory
# then get path to draft business listings file
listingsPath  = os.path.join(parentDir,'businessListings\draftBusinessListings.csv')

listings = pd.read_csv(listingsPath).drop(columns='Unnamed: 0')

Dataframe to geopandas

In [47]:
listings = gpd.GeoDataFrame(listings, 
                      geometry=gpd.points_from_xy(listings.long,listings.lat)
                      )

In [48]:
listings = listings.assign(tract = np.nan)

# checking if coordinates belong to any of the census tract polygon (see tracts dataframe)
for i in range(len(tracts)):
  # for each coordinate, is it located within x tract
  mask = listings.within(tracts['geometry'].iloc[i])
    
  # must be unique (matched one time)
  if len(listings.loc[mask,'tract'].unique())>1: 
    raise AssertionError('Row already has an assigned Tract Name. May lead to duplicates.')
    
  # assign tract name to dummy column where matches ocurred
  listings.loc[mask,'tract'] = tracts['TRACTCE'].iloc[i]
    
listings.to_csv('FINALBusListings.csv')

In [53]:
(listings.isnull().sum() / listings.shape[0]).multiply(100).round(2)

account_num             0.00
business_name           0.00
street_address          0.00
city                    0.00
zip_code                0.00
location_start_date     0.00
location_end_date      89.89
lat                     0.00
long                    0.00
geometry                0.00
tract                   0.20
dtype: float64

After our check, around 0.20% of the rows did not match to any census tract. In our project these rows where dropped as they were relatively a small portion of businesses and thus would not have a big impact on our project.

## Caveats on the zip code to census tract conversion: There were some census tracts who shape boundary did not completely crossed a zip code-- ie. half of the shape was inside the zip code boundary the other half wasn't.  This is another limitation of our project and should be addressed in future iterations of this project. 